# Analysis of the results from part 1 of model training

In [1]:
from typing import cast
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# initialise spark session
spark = cast(SparkSession, SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Team4-Project-p2-analysis") \
    .getOrCreate()
)

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/12 12:07:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/12 12:07:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/12 12:07:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
predictions2 = spark.read.load('../data/predictions2')

In [3]:
predictions2.printSchema()

root
 |-- FID: string (nullable = true)
 |-- PHENOTYPE: integer (nullable = true)
 |-- prediction: double (nullable = true)



### Confusion matrix, accuracy, precision, recall

In [4]:
# pyspark only supports confusion matrix for multiclass classification, and only for rdd-based API, so need to convert it to RDD first
# reference: https://stackoverflow.com/questions/58404845/confusion-matrix-to-get-precsion-recall-f1score

from pyspark import RDD
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType

predictions_labels = cast(
    RDD[tuple[float, float]],
    predictions2 \
        .select(F.col('PHENOTYPE').cast(FloatType()), 'prediction') \
        .rdd.map(tuple)
)

metrics = MulticlassMetrics(predictions_labels)

print('confusion matrix: ')
print(metrics.confusionMatrix().toArray())
print(f'accuracy: {metrics.accuracy}')
print(f'precision: {metrics.precision(1)}')
print(f'recall: {metrics.recall(1)}')


/root/miniconda3/envs/pyspark_env/lib/python3.11/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


confusion matrix: 
[[ 38.   6.]
 [  9. 111.]]
accuracy: 0.9085365853658537
precision: 0.9487179487179487
recall: 0.925
